In [27]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent,create_tool_calling_agent
from langchain.tools import tool
from langchain_core.callbacks import Callbacks
from langchain.prompts import ChatPromptTemplate
import random
import pprint

In [2]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

<p>Streaming with Agents are made more complicated by the fact that it's not just tokens of the final answer that you will want to stream, but you may also want to stream back the intermediate steps an agent takes</p>

In [47]:
@tool
async def whereCatisHiding() -> str:
    """
        Where the cat is hiding right now?
    """
    return random.choice(seq=["On the Bed","On the Shelf"])

In [48]:
@tool
async def getItems(place:str) -> str:
    """
        Use this tool to look up which items are in the given place
    """
    if "bed" in place:  # For under the bed
        return "socks, shoes and dust bunnies"
    elif "shelf" in place:  # For shelf
        return "books, penciles and pictures"
    else:  # if the agent decides to ask about a different place
        return "cat snacks"

In [49]:
await whereCatisHiding.ainvoke(input={})

'On the Bed'

In [50]:
await getItems.ainvoke(input={"place":"almirah"})

'cat snacks'

<h3>Initialize the Agent</h3>

In [51]:
prompt=hub.pull(owner_repo_commit="hwchase17/openai-tools-agent")

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [52]:
tools=[getItems,whereCatisHiding]

In [53]:
agent=create_openai_tools_agent(
    llm=llmOpenAI.with_config({"tags":["agent_llm"]}),
    tools=tools,
    prompt=prompt
)

In [54]:
agentExecutor=AgentExecutor(agent=agent,tools=tools).\
    with_config({"run_name":"Agent"})  # this config will be used later with astream_events API

<h3> Stream Intermediate Steps</h3>
<h4>Actions, Observations and Final Answer</h4>

In [55]:
chunks=[]
async for chunk in agentExecutor.astream(input={"input":"What items are located where the cat is hiding?"}):
    chunks.append(chunk)
    print("---------")
    pprint.pprint(object=chunk,depth=1)

---------
{'actions': [...], 'messages': [...]}
---------
{'messages': [...], 'steps': [...]}
---------
{'actions': [...], 'messages': [...]}
---------
{'messages': [...], 'steps': [...]}
---------
{'messages': [...],
 'output': 'The items located where the cat is hiding (on the bed) are cat '
           'snacks.'}


In [62]:
chunks

[{'actions': [ToolAgentAction(tool='whereCatisHiding', tool_input={}, log='\nInvoking: `whereCatisHiding` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_vwqj6ZdL69MdhAQjZADBXUW4', 'function': {'arguments': '{}', 'name': 'whereCatisHiding'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-430e0c96-1ea7-426d-a638-b912af21ce51', tool_calls=[{'name': 'whereCatisHiding', 'args': {}, 'id': 'call_vwqj6ZdL69MdhAQjZADBXUW4', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'whereCatisHiding', 'args': '{}', 'id': 'call_vwqj6ZdL69MdhAQjZADBXUW4', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_vwqj6ZdL69MdhAQjZADBXUW4')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_vwqj6ZdL69MdhAQjZADBXUW4', 'function': {'arguments': '{}', 'name': 'whereCatisHiding'}, 'type': 'function'}]}, response_metad

In [61]:
for chunk in chunks:
    print(chunk['messages'])

[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_vwqj6ZdL69MdhAQjZADBXUW4', 'function': {'arguments': '{}', 'name': 'whereCatisHiding'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-430e0c96-1ea7-426d-a638-b912af21ce51', tool_calls=[{'name': 'whereCatisHiding', 'args': {}, 'id': 'call_vwqj6ZdL69MdhAQjZADBXUW4', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'whereCatisHiding', 'args': '{}', 'id': 'call_vwqj6ZdL69MdhAQjZADBXUW4', 'index': 0, 'type': 'tool_call_chunk'}])]
[FunctionMessage(content='On the Bed', additional_kwargs={}, response_metadata={}, name='whereCatisHiding')]
[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ZvrSfkVXgp5I2q4BlqtRwDRe', 'function': {'arguments': '{"place":"On the Bed"}', 'name': 'getItems'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='r

<h3>Using AgentAction/Observation</h3>

In [66]:
async for chunk in agentExecutor.astream(input={"input":"What items are located where the cat is hiding?"}):
    # Agent Action
    if "actions" in chunk:
        for action in chunk['actions']:
            print(f"Calling Tool: {action.tool} with Input: {action.tool_input}")
    # Observation
    elif "steps" in chunk:
        for step in chunk["steps"]:
            print(f"Tool Result: {step.observation}")
    # Final Result
    elif "output" in chunk:
        print(f"Final Output: {chunk['output']}")
    else:
        raise ValueError
    print("----")
    
            
        

Calling Tool: whereCatisHiding with Input: {}
----
Tool Result: On the Shelf
----
Calling Tool: getItems with Input: {'place': 'On the Shelf'}
----
Tool Result: cat snacks
----
Final Output: The items located where the cat is hiding (On the Shelf) are cat snacks.
----


<h3> Custom Streaming with Events</h3>

In [67]:
async for chunk in agentExecutor.astream_events(
    input={"input":"What items are located where the cat is hiding?"},
    version="v2"
    ):
    print(chunk)

{'event': 'on_chain_start', 'data': {'input': {'input': 'What items are located where the cat is hiding?'}}, 'name': 'Agent', 'tags': [], 'run_id': '81c1794c-706f-4eb6-be41-f8b1a583d798', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {}, 'name': 'RunnableSequence', 'tags': [], 'run_id': '8ed945bf-ba92-4406-b0eb-dc4b3be8fab4', 'metadata': {}, 'parent_ids': ['81c1794c-706f-4eb6-be41-f8b1a583d798']}
{'event': 'on_chain_start', 'data': {}, 'name': 'RunnableAssign<agent_scratchpad>', 'tags': ['seq:step:1'], 'run_id': 'e8142567-5632-48a9-b519-402b243f27e8', 'metadata': {}, 'parent_ids': ['81c1794c-706f-4eb6-be41-f8b1a583d798', '8ed945bf-ba92-4406-b0eb-dc4b3be8fab4']}
{'event': 'on_chain_stream', 'run_id': 'e8142567-5632-48a9-b519-402b243f27e8', 'name': 'RunnableAssign<agent_scratchpad>', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {'chunk': {'input': 'What items are located where the cat is hiding?', 'intermediate_steps': []}}, 'parent_ids': ['81c1794c-706f-4eb6-b

In [70]:
async for event in agentExecutor.astream_events(
    {"input": "where is the cat hiding? what items are in that location?"},
    version="v2",
):
    kind=event["event"]
    if kind=="on_chain_start":
        if (
            event["name"]=="Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind=="on_chain_end":
        if (
            event["name"]=="Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind=="on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind=="on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind=="on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: Agent with input: {'input': 'where is the cat hiding? what items are in that location?'}
--
Starting tool: whereCatisHiding with inputs: {}
Done tool: whereCatisHiding
Tool output was: On the Shelf
--
--
Starting tool: getItems with inputs: {'place': 'On the Shelf'}
Done tool: getItems
Tool output was: cat snacks
--
The| cat| is| hiding| on| the| shelf|,| and| the| items| in| that| location| are| cat| snacks|.|
--
Done agent: Agent with output: The cat is hiding on the shelf, and the items in that location are cat snacks.


<p> If your tool leverages Langchain runnable objects, (eg. LCEL Chains, LLMs, retrievers, etc) and you want to stream events from those objects as well, you need to make sure that callbacks are propagated correctly</p> 

<h3>Stream Events from within Tools</h3>

In [74]:
@tool
async def getItems(place: str, callbacks:Callbacks) -> str:
    """
        Use this tool to look up which items are in the given place
    """
    template=ChatPromptTemplate.from_messages(
        messages=[
            (
                "human",
                "Can you tell me what kind of items I might find in the following place: {place}"
                "List at least 3 such items separating them by coma. And include a brief description of each item.."
            )
        ]
    )
    
    chain=template|llmOpenAI.with_config(config={
        "run_name":"Get Items LLM",
        "tags":["tool_llm"],
        "callbacks":callbacks
    })

    chunks=[chunk async for chunk in chain.astream(input={"place":place})]
    return "".join(chunk.content for chunk in chunks)

In [76]:
prompt=hub.pull(owner_repo_commit="hwchase17/openai-tools-agent")

tools=[getItems,whereCatisHiding]

agent=create_openai_tools_agent(
    llm=llmOpenAI.with_config({"tags":["agent_llm"]}),
    tools=tools,
    prompt=prompt
)

agentExecutor=AgentExecutor(agent=agent,tools=tools).\
    with_config({"run_name":"Agent"})  # this config will be used later with astream_events API


C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [78]:
async for event in agentExecutor.astream_events(
    {"input": "where is the cat hiding? what items are in that location?"},
    version="v2",
):
    kind=event["event"]
    if kind=="on_chain_start":
        if (
            event["name"]=="Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind=="on_chain_end":
        if (
            event["name"]=="Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind=="on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind=="on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind=="on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: Agent with input: {'input': 'where is the cat hiding? what items are in that location?'}
--
Starting tool: whereCatisHiding with inputs: {}
Done tool: whereCatisHiding
Tool output was: On the Bed
--
--
Starting tool: getItems with inputs: {'place': 'On the Bed'}
1|.| Pill|ows| -| Soft| cushions| used| for| supporting| the| head| or| other| parts| of| the| body| while| sleeping| or| resting|.

|2|.| Blank|ets| -| Large| pieces| of| cloth| used| for| keeping| warm| while| sleeping|,| typically| made| of| wool| or| fleece|.

|3|.| St|uffed| animals| -| Soft| toys| in| the| shape| of| animals|,| often| used| for| decoration| or| as| comforting| companions| while| sleeping|.|Done tool: getItems
Tool output was: 1. Pillows - Soft cushions used for supporting the head or other parts of the body while sleeping or resting.

2. Blankets - Large pieces of cloth used for keeping warm while sleeping, typically made of wool or fleece.

3. Stuffed animals - Soft toys in the shape of a

<h3> Using AStream Log </h3> <h4>(Recommended to use Astream Log instead of this for Parsing problem)</h4>

In [72]:
i=0
async for chunk in agentExecutor.astream_log(
    input={"input":"Where is the cat hiding? What items are in that location?"}
    ):
    print(chunk)
    i+=1
    if i>20: break

RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': 'adcb0b2e-6ad6-4e6c-adba-86a89c6a9a75',
            'logs': {},
            'name': 'Agent',
            'streamed_output': [],
            'type': 'chain'}})
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableSequence',
  'value': {'end_time': None,
            'final_output': None,
            'id': '608bf962-aae1-4c52-aa5e-2703ed69e0d3',
            'metadata': {},
            'name': 'RunnableSequence',
            'start_time': '2024-11-01T14:16:55.248+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': [],
            'type': 'chain'}})
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableAssign<agent_scratchpad>',
  'value': {'end_time': None,
            'final_output': None,
            'id': '0b4d099a-bcc3-4d54-8b7a-3098bc8160ee',
            'metadata': {},
            'name': 'RunnableAssign<agent_scratchpad>',
            'start_ti